In [2]:
import numpy as np
from scipy.misc import imresize
import pylab
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import transform, filters
import sys, os

%matplotlib notebook

%load_ext autoreload
%autoreload 2

import caffe

from helpers import *
from heatmaps import *
from optimize_mask import *

In [4]:
# TODO: Change this to your own Caffe-ExcitationBP directory
caffe_dir = '/home/ruthfong/sample_code/Caffe-ExcitationBP/'

gpu = 0 # gpu = None # for CPU-only
net_type = 'googlenet'

if gpu is not None:
    caffe.set_device(gpu)
    caffe.set_mode_gpu()
else:
    caffe.set_mode_cpu()

net = get_net(net_type)

In [19]:
labels_desc = np.loadtxt(os.path.join(caffe_dir, 'data/ilsvrc12/synset_words.txt'), str, delimiter='\t')
synsets = np.loadtxt(os.path.join(caffe_dir, 'data/ilsvrc12/synsets.txt'), str, delimiter='\t')
indexing = np.loadtxt('../../../data/ilsvrc12/ascii_order_to_synset_order.txt')
reverse_indexing = np.loadtxt('../../../data/ilsvrc12/synset_order_to_ascii_order.txt', dtype=int)
(paths, labels) = read_imdb('../../../data/ilsvrc12/annotated_train_heldout_imdb.txt')
data_desc = 'train_heldout'

# Set generate_ann_dir = True to create a dir containing only the 5000 heldout training images' bounding box annotations (one-time only)
# Download ImageNet bounding box here: http://image-net.org/Annotation/Annotation.tar.gz 
# (from http://image-net.org/download-bboxes)
# TODO set orig_ilsvrc_ann_dir to the unzipped Annotation dir and ann_dir to where you want to save the heldout ann dir
generate_ann_dir = False
if generate_ann_dir:
    orig_ilsvrc_ann_dir = '/data/ruthfong/ILSVRC2012/Annotation'
    train_ann_dir = '/data/ruthfong/ILSVRC2012/annotated_train_heldout_ground_truth_annotations'
    create_heldout_annotated_dir(orig_ilsvrc_ann_dir, ann_dir, 
                                 imdb='../../../data/ilsvrc12/annotated_train_heldout_imdb.txt')
else:
    train_ann_dir = '/data/ruthfong/ILSVRC2012/annotated_train_heldout_ground_truth_annotations'
ann_dir = train_ann_dir
ann_paths = np.sort([os.path.join(ann_dir, f) for f in os.listdir(ann_dir)])

# TODO - set to the validation annotations dir
val_ann_dir = '/data/ruthfong/ILSVRC2012/val'

# TODO replace with mask dir (ideally holding learned masks from the 5000 heldout training images)
# though most examples can be generated in this notebook, in which case make mask_dir the location in which
# you want to save the learned masks and set generate_masks = True
# Note: for localization, we use slightly different settings
generate_masks = False
mask_dir = '/data/ruthfong/neural_coding/pycaffe_results/googlenet_train_heldout_given_grad_1_norm_0/min_top5_prob_blur/lr_-1.00_l1_lambda_-3.00_tv_lambda_-inf_l1_lambda_2_-2.00_beta_2.00_mask_scale_8_blur_mask_5_jitter_4_noise_-inf_num_iters_300_tv2_mask_init'
# mask_dir = '/data/ruthfong/neural_coding/pycaffe_results/googlenet_train_heldout_given_grad_1_norm_0/min_top0_prob_blur/lr_-1.00_l1_lambda_-4.00_tv_lambda_-inf_l1_lambda_2_-2.00_beta_3.00_mask_scale_8_blur_mask_5_jitter_4_noise_-inf_num_iters_300_tv2_mask_init'
if generate_masks == False:
    mask_paths = [os.path.join(mask_dir, '%d.npy' % x) for x in range(len(labels))]

## Pre-reqs
* Learn optimized masks for 50k validation images using localization hyperparameters (Run `python optimize_mask.py val localization`)
* Run `python heatmaps.py OUT_PATH val HEATMAP_TYPE THRESHOLD_TYPE [MASK_DIR] [GPU]`
    * `OUT_PATH` is where to save the output file with bounding box information
    * `MASK_DIR` is where the masks for the 50k val images are saved (only include when using `HEATMAP_TYPE` = `mask`), 
    * `HEATMAP_TYPE` $\in$ [`mask`, `saliency` (a.k.a. gradient), `guided_backprop`, `excitation_backprop`], and 
    * `THRESHOLD_TYPE` $\in$ [`min_max_diff` (a.k.a. value), `energy`, `mean`]
* (Optional, to replicate finding alpha thresholds) Learn optimized masks for 5000 heldout images using localization hyperparameters (Run `python optimize_mask.py heldout localization`)
* (Optional, to replicate finding alpha thresholds) Run `python heatmaps.py OUT_DIR annotated_train_heldout HEATMAP_TYPE THRESHOLD_TYPE [MASK_DIR] [GPU]`
    * `OUT_DIR` is the directory in which to save the output files with bounding box information for the alpha search
    * `MASK_DIR` is where the masks for the 5k heldout training images are saved (only include when using `HEATMAP_TYPE` = `mask`), 

In [13]:
def find_best_alpha(bb_file_prefix, alphas = np.arange(0,1,0.05)):
    errs = np.zeros(len(alphas))
    for i in range(len(alphas)):
        alpha = alphas[i]
        bb_file = bb_file_prefix % alpha
        (err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets, 
                                                   reverse_indexing=reverse_indexing)
        errs[i] = err
    for i in range(len(alphas)):
        print 'alpha = %.2f, err = %f' % (alphas[i], errs[i])
    min_i = np.argmin(errs)
    print 'best alpha = %.2f, err = %f' % (alphas[min_i], errs[min_i])
    return errs

# Energy-Alpha

## Finding Best Alpha (optional)

In [16]:
ann_dir = train_ann_dir
ann_paths = np.sort([os.path.join(ann_dir, f) for f in os.listdir(ann_dir)])

heatmap_types = ['Mask', 'Gradient', 'Guided Backprop', 'Excitation Backprop']
mask_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/googlenet_train_heldout_given_grad_1_norm_0/min_top5_prob_blur/lr_-1.00_l1_lambda_-3.00_tv_lambda_-inf_l1_lambda_2_-2.00_beta_2.00_mask_scale_8_blur_mask_5_jitter_4_noise_-inf_num_iters_300_tv2_mask_init/energy_alpha_%.2f_norm_Inf.txt'
gradient_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/saliency/energy_alpha_%.2f_norm_Inf.txt'
guided_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/guided_backprop/energy_alpha_%.2f_norm_Inf.txt'
exc_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/excitation_backprop/energy_alpha_%.2f_norm_-1.txt'
formats = [mask_trn_loc_format, gradient_trn_loc_format, guided_trn_loc_format, exc_trn_loc_format]

assert (len(formats) == len(heatmap_types))
for i in range(len(formats)):
    print heatmap_types[i]
    _ = find_best_alpha(formats[i])
    print ''

Mask
alpha = 0.00, err = 0.790000
alpha = 0.05, err = 0.912600
alpha = 0.10, err = 0.861800
alpha = 0.15, err = 0.822400
alpha = 0.20, err = 0.785400
alpha = 0.25, err = 0.750000
alpha = 0.30, err = 0.722400
alpha = 0.35, err = 0.693800
alpha = 0.40, err = 0.671000
alpha = 0.45, err = 0.638200
alpha = 0.50, err = 0.610800
alpha = 0.55, err = 0.586800
alpha = 0.60, err = 0.558800
alpha = 0.65, err = 0.528800
alpha = 0.70, err = 0.494800
alpha = 0.75, err = 0.470400
alpha = 0.80, err = 0.442600
alpha = 0.85, err = 0.417200
alpha = 0.90, err = 0.396000
alpha = 0.95, err = 0.391200
best alpha = 0.95, err = 0.391200
Gradient
alpha = 0.00, err = 0.790000
alpha = 0.05, err = 0.458000
alpha = 0.10, err = 0.402600
alpha = 0.15, err = 0.405600
alpha = 0.20, err = 0.419200
alpha = 0.25, err = 0.438200
alpha = 0.30, err = 0.452000
alpha = 0.35, err = 0.465600
alpha = 0.40, err = 0.479000
alpha = 0.45, err = 0.495200
alpha = 0.50, err = 0.507200
alpha = 0.55, err = 0.518600
alpha = 0.60, err = 0.52

## Testing energy-alpha on validation set

In [20]:
ann_dir = val_ann_dir
ann_paths = np.sort([os.path.join(ann_dir, f) for f in os.listdir(ann_dir)])

In [21]:
# TODO - change to output file from heatmaps.py
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/googlenet_val_given_grad_1_norm_0/min_top5_prob_blur/lr_-1.00_l1_lambda_-3.00_tv_lambda_-inf_l1_lambda_2_-2.00_beta_2.00_mask_scale_8_blur_mask_5_jitter_4_noise_-inf_num_iters_300_tv2_mask_init/energy_alpha_0.95_norm_Inf.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'mask', err

mask 0.4425


In [13]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/saliency/energy_alpha_0.10_norm_Inf.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'gradient', err

saliency 0.44946


In [16]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/guided_backprop/energy_alpha_0.30_norm_Inf.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'guided_backprop', err

guided_backprop 0.47962


In [6]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/excitation_backprop/energy_alpha_0.60_norm_-1.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'excitation_backprop', err

excitation_backprop 0.3992


# Value-Alpha
## Finding Best Alpha (optional)

In [17]:
ann_dir = train_ann_dir
ann_paths = np.sort([os.path.join(ann_dir, f) for f in os.listdir(ann_dir)])

heatmap_types = ['Mask', 'Gradient', 'Guided Backprop', 'Excitation Backprop']
mask_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/googlenet_train_heldout_given_grad_1_norm_0/min_top5_prob_blur/lr_-1.00_l1_lambda_-3.00_tv_lambda_-inf_l1_lambda_2_-2.00_beta_2.00_mask_scale_8_blur_mask_5_jitter_4_noise_-inf_num_iters_300_tv2_mask_init/min_max_diff_alpha_%.2f_norm_Inf.txt'
gradient_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/saliency/min_max_diff_alpha_%.2f_norm_Inf.txt'
guided_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/guided_backprop/min_max_diff_alpha_%.2f_norm_Inf.txt'
exc_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/excitation_backprop/min_max_diff_alpha_%.2f_norm_-1.txt'
formats = [mask_trn_loc_format, gradient_trn_loc_format, guided_trn_loc_format, exc_trn_loc_format]

assert (len(formats) == len(heatmap_types))
for i in range(len(formats)):
    print heatmap_types[i]
    _ = find_best_alpha(formats[i])
    print ''

Mask
alpha = 0.00, err = 0.553000
alpha = 0.05, err = 0.419400
alpha = 0.10, err = 0.402400
alpha = 0.15, err = 0.406000
alpha = 0.20, err = 0.408600
alpha = 0.25, err = 0.424000
alpha = 0.30, err = 0.440800
alpha = 0.35, err = 0.465000
alpha = 0.40, err = 0.481600
alpha = 0.45, err = 0.506400
alpha = 0.50, err = 0.528200
alpha = 0.55, err = 0.554600
alpha = 0.60, err = 0.583000
alpha = 0.65, err = 0.606000
alpha = 0.70, err = 0.635800
alpha = 0.75, err = 0.666200
alpha = 0.80, err = 0.703600
alpha = 0.85, err = 0.743600
alpha = 0.90, err = 0.795800
alpha = 0.95, err = 0.877800
best alpha = 0.10, err = 0.402400

Gradient
alpha = 0.00, err = 0.556200
alpha = 0.05, err = 0.533400
alpha = 0.10, err = 0.481400
alpha = 0.15, err = 0.437000
alpha = 0.20, err = 0.424000
alpha = 0.25, err = 0.423400
alpha = 0.30, err = 0.463200
alpha = 0.35, err = 0.514600
alpha = 0.40, err = 0.583600
alpha = 0.45, err = 0.652800
alpha = 0.50, err = 0.723200
alpha = 0.55, err = 0.787800
alpha = 0.60, err = 0.8

## Testing value-alpha on validation set

In [23]:
ann_dir = val_ann_dir
ann_paths = np.sort([os.path.join(ann_dir, f) for f in os.listdir(ann_dir)])

In [24]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/googlenet_val_given_grad_1_norm_0/min_top5_prob_blur/lr_-1.00_l1_lambda_-3.00_tv_lambda_-inf_l1_lambda_2_-2.00_beta_2.00_mask_scale_8_blur_mask_5_jitter_4_noise_-inf_num_iters_300_tv2_mask_init/min_max_diff_alpha_0.10_norm_Inf.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'mask', err

mask 0.45104


In [25]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/saliency/min_max_diff_alpha_0.25_norm_Inf.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'gradient', err

gradient 0.4704


In [26]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/guided_backprop/min_max_diff_alpha_0.05_norm_Inf.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'guided_backprop', err

guided_backprop 0.5118


In [27]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/excitation_backprop/min_max_diff_alpha_0.15_norm_-1.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'excitation_backprop', err

excitation_backprop 0.4711


# Mean Alpha
## Finding Best Alpha (optional)

In [ ]:
ann_dir = train_ann_dir
ann_paths = np.sort([os.path.join(ann_dir, f) for f in os.listdir(ann_dir)])

alphas = np.arange(0, 10.5, 0.5)
heatmap_types = ['Mask', 'Gradient', 'Guided Backprop', 'Excitation Backprop']
mask_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/googlenet_train_heldout_given_grad_1_norm_0/min_top5_prob_blur/lr_-1.00_l1_lambda_-3.00_tv_lambda_-inf_l1_lambda_2_-2.00_beta_2.00_mask_scale_8_blur_mask_5_jitter_4_noise_-inf_num_iters_300_tv2_mask_init/alpha_%.1f_norm_Inf.txt'
gradient_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/saliency/alpha_%.1f_norm_Inf.txt'
guided_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/guided_backprop/alpha_%.1f_norm_Inf.txt'
exc_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/excitation_backprop/alpha_%.1f_norm_-1.txt'
formats = [mask_trn_loc_format, gradient_trn_loc_format, guided_trn_loc_format, exc_trn_loc_format]

assert (len(formats) == len(heatmap_types))
for i in range(len(formats)):
    print heatmap_types[i]
    _ = find_best_alpha(formats[i], alphas = alphas)
    print ''

In [24]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/googlenet_val_given_grad_1_norm_0/min_top5_prob_blur/lr_-1.00_l1_lambda_-3.00_tv_lambda_-inf_l1_lambda_2_-2.00_beta_2.00_mask_scale_8_blur_mask_5_jitter_4_noise_-inf_num_iters_300_tv2_mask_init/min_max_diff_alpha_0.10_norm_Inf.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'mask', err

mask 0.45104


In [25]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/saliency/min_max_diff_alpha_0.25_norm_Inf.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'gradient', err

gradient 0.4704


In [ ]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/guided_backprop/min_max_diff_alpha_0.05_norm_Inf.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'guided_backprop', err

guided_backprop 0.5118


In [ ]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/excitation_backprop/min_max_diff_alpha_0.15_norm_-1.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'excitation_backprop', err

excitation_backprop 0.4711


# Mean Alpha
## Finding Best Alpha (optional)

In [ ]:
ann_dir = train_ann_dir
ann_paths = np.sort([os.path.join(ann_dir, f) for f in os.listdir(ann_dir)])

alphas = np.arange(0, 10.5, 0.5)
heatmap_types = ['Mask', 'Gradient', 'Guided Backprop', 'Excitation Backprop']
mask_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/googlenet_train_heldout_given_grad_1_norm_0/min_top5_prob_blur/lr_-1.00_l1_lambda_-3.00_tv_lambda_-inf_l1_lambda_2_-2.00_beta_2.00_mask_scale_8_blur_mask_5_jitter_4_noise_-inf_num_iters_300_tv2_mask_init/alpha_%.1f_norm_Inf.txt'
gradient_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/saliency/alpha_%.1f_norm_Inf.txt'
guided_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/guided_backprop/alpha_%.1f_norm_Inf.txt'
exc_trn_loc_format = '/data/ruthfong/neural_coding/loc_preds/googlenet_annotated_train_heldout_gt_pycaffe/excitation_backprop/alpha_%.1f_norm_-1.txt'
formats = [mask_trn_loc_format, gradient_trn_loc_format, guided_trn_loc_format, exc_trn_loc_format]

assert (len(formats) == len(heatmap_types))
for i in range(len(formats)):
    print heatmap_types[i]
    _ = find_best_alpha(formats[i], alphas = alphas)
    print ''

## Testing mean-alpha on validation set

In [28]:
ann_dir = val_ann_dir
ann_paths = np.sort([os.path.join(ann_dir, f) for f in os.listdir(ann_dir)])

In [29]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/googlenet_val_given_grad_1_norm_0/min_top5_prob_blur/lr_-1.00_l1_lambda_-3.00_tv_lambda_-inf_l1_lambda_2_-2.00_beta_2.00_mask_scale_8_blur_mask_5_jitter_4_noise_-inf_num_iters_300_tv2_mask_init/alpha_0.5_norm_Inf.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'mask', err

mask 0.44342


In [30]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/saliency/alpha_5.0_norm_Inf.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'gradient', err

IOError: [Errno 2] No such file or directory: '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/saliency/alpha_5.0_norm_Inf.txt'

In [ ]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/guided_backprop/alpha_4.5_norm_Inf.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'guided_backprop', err

In [ ]:
bb_file = '/data/ruthfong/neural_coding/loc_preds/googlenet_val_gt_pycaffe/excitation_backprop/alpha_1.5_norm_-1.txt'
(err, _, _) = compute_localization_results(bb_file, ann_paths, synsets=synsets,
                             reverse_indexing=reverse_indexing)
print 'excitation_backprop', err